In [ ]:
!nvidia-smi

Tue Feb  1 17:08:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v3"

/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v4


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 12.2 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
     |████████████████████████████████| 6.8 MB 27.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import json
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
# Get device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
!ls results/checkpoint-12000

ls: cannot access 'results/checkpoint-12000': No such file or directory


In [ ]:
model_path = "results/checkpoint-450"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2).to(device)
test_trainer = Trainer(model)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
#remove punctuation
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

table = str.maketrans('', '', string.punctuation)

# Remove stop words
from nltk.corpus import stopwords

stop = stopwords.words('english')

# Lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
maybe_bad_tweets = {}
maybe_bad_tweets["tweets"] = []
for i in range(1000):
    try:
        followers_timeline_file = open("../followers_timeline_{}.json".format(str(i)), 'r')
        print("Working on file {}".format(str(i)))
    except:
        print("File not found")
        break
    followers_timeline = json.load(followers_timeline_file)
    tweet_text_list = []
    tweets_list = []
    for user in followers_timeline["followers_timeline"]:
        for follower in followers_timeline["followers_timeline"][user]:
            try:
                tweet_timeline = followers_timeline["followers_timeline"][user][follower]["data"]
                
                for tweet in tweet_timeline:
                        tweet_text = tweet['text']
                        tweet_text = ' '.join([word for word in tweet_text.lower().split() if word not in stop])  # remove stopwords
                        # remove punc.
                        tweet_text = tweet_text.translate(table)
                        tweet_text = ''.join(c for c in tweet_text if not c.isdigit())  # remove numbers
                        tweet_text = tweet_text.replace('  ', ' ')  # remove double spaces
                        tweet_text = ' '.join([lemmatizer.lemmatize(word) for word in tweet_text.split()])  # lemminize
                        tweet_text_list.append(tweet_text)
                        tweets_list.append(tweet)
            except:
                pass

    X_test_tokenized = tokenizer(tweet_text_list, padding=True, truncation=True, max_length=512)
    test_dataset = Dataset(X_test_tokenized)
    raw_pred, tmp1, tmp2 = test_trainer.predict(test_dataset)

    y_prob = tf.math.softmax(raw_pred, axis=-1).numpy()
    y_pred = np.argmax(raw_pred, axis=1)
    counter = 0
    for tweet_pred in y_prob:
        # print(tweet_pred[1])
        if tweet_pred[1] >= np.float32(0.95):
            print(tweet_text_list[counter])
            print(type(tweet_pred[1]))
            print(tweet_pred[1])
            tweets_list[counter]["confidence"] = tweet_pred[1].item()
            maybe_bad_tweets["tweets"].append(tweets_list[counter])
        counter += 1
                        
            

Working on file 0


***** Running Prediction *****
  Num examples = 8196
  Batch size = 8


gorgeous sculpture henry moore great setting httpstcomssvsqkp
<class 'numpy.float32'>
0.9786227
another beautiful piece sale brand new affordable part online glass selling exhibition at httpstcongwgfkguy purchase please contact artist directly find many amazing unique piece glass sale stunning show httpstcovlxcrcjnw
<class 'numpy.float32'>
0.993383
canadian clay amp glass building registry artist specialize clay glass amp copper enamel art information gathered shared canadian clay glass gallery future exhibition amp collaborative opportunity httpstcoavmmfzclc artstartsto httpstcovycutegv
<class 'numpy.float32'>
0.956436
three wise hare cm seenohearnospeakno evil bronze hare figurine decoration decor besthomedecor gift giftideas perfectforgift httpstcoysudhpvw
<class 'numpy.float32'>
0.98747534
angel offering cm white angel offering kneeling caped angel figurine besthomedecor homedecor decor decoration gift perfectgift giftideas httpstcommpmfqfsl
<class 'numpy.float32'>
0.96241134
bapho

***** Running Prediction *****
  Num examples = 8067
  Batch size = 8


KeyboardInterrupt: ignored

In [ ]:
# Sort by confidence
maybe_bad_tweets["tweets"] = sorted(maybe_bad_tweets["tweets"], key=lambda k: k['confidence'], reverse=True)

In [ ]:
# Write maybe bad tweets to file
with open("maybe_bad_tweets_v6.json", 'w') as outfile:
	json.dump(maybe_bad_tweets, outfile)